<a href="https://colab.research.google.com/github/HoussemDs/patient-hospital/blob/main/Patient_Classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdulqaderasiirii/hospital-patient-data")

print("Path to dataset files:", path)

100%|██████████| 2.04M/2.04M [00:00<00:00, 93.3MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/abdulqaderasiirii/hospital-patient-data/versions/1


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier



In [ ]:
import os

files = os.listdir(path)
print("Files in directory:", files)


Files in directory: ['hospital_data_sampleee.xlsx']


In [ ]:
file_name = "hospital_data_sampleee.xlsx"
file_path = os.path.join(path, file_name)
df = pd.read_excel(file_path)

df.head()

,Date,Medication Revenue,Lab Cost,Consultation Revenue,Doctor Type,Financial Class,Patient Type,Entry Time,Post-Consultation Time,Completion Time,Patient ID
0,2019-11-04,1183.22,10,20.17,ANCHOR,HMO,OUTPATIENT,08:35:45,09:17:54,09:29:46,C10001
1,2019-11-06,738.48,$-,15,ANCHOR,INSURANCE,OUTPATIENT,19:19:16,21:02:36,21:24:07,C10002
2,2019-11-02,660,$-,21.17,ANCHOR,HMO,OUTPATIENT,10:46:52,11:56:25,12:06:28,C10003
3,2019-11-06,600,$-,$-,ANCHOR,MEDICARE,OUTPATIENT,09:38:34,10:55:50,10:58:02,C10004
4,2019-11-01,591.6,$-,12,ANCHOR,INSURANCE,OUTPATIENT,11:16:21,12:06:49,12:06:54,C10005


In [ ]:
df.describe()

,Date
count,29998
mean,2019-11-07 00:06:20.185345536
min,2019-11-01 00:00:00
25%,2019-11-04 00:00:00
50%,2019-11-07 00:00:00
75%,2019-11-11 00:00:00
max,2019-11-13 00:00:00


In [ ]:
df.describe(include='all')

,Date,Medication Revenue,Lab Cost,Consultation Revenue,Doctor Type,Financial Class,Patient Type,Entry Time,Post-Consultation Time,Completion Time,Patient ID
count,29998,29998,29998,29998,29998,29998,29998,29998,29998,29998,29998
unique,NaN,4280,200,241,3,5,1,20628,21356,21384,29998
top,NaN,$-,$-,$-,ANCHOR,INSURANCE,OUTPATIENT,18:07:47,11:44:23,10:04:33,C10001
freq,NaN,11936,28565,5576,21913,9931,29998,9,6,7,1
mean,2019-11-07 00:06:20.185345536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2019-11-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2019-11-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2019-11-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2019-11-11 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2019-11-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# see the type of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29998 entries, 0 to 29997
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    29998 non-null  datetime64[ns]
 1    Medication Revenue     29998 non-null  object        
 2     Lab Cost              29998 non-null  object        
 3    Consultation Revenue   29998 non-null  object        
 4   Doctor Type             29998 non-null  object        
 5   Financial Class         29998 non-null  object        
 6   Patient Type            29998 non-null  object        
 7   Entry Time              29998 non-null  object        
 8   Post-Consultation Time  29998 non-null  object        
 9   Completion Time         29998 non-null  object        
 10  Patient ID              29998 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 2.5+ MB


In [ ]:
df.columns = df.columns.str.strip()
df.columns

Index(['Date', 'Medication Revenue', 'Lab Cost', 'Consultation Revenue',
       'Doctor Type', 'Financial Class', 'Patient Type', 'Entry Time',
       'Post-Consultation Time', 'Completion Time', 'Patient ID'],
      dtype='object')

In [ ]:
# changing the type of the columns
df['Medication Revenue'].value_counts(normalize=True)

,proportion
Medication Revenue,
$-,0.397893
7.48,0.004967
5,0.004300
17,0.004200
8.5,0.003934
...,...
56.62,0.000033
56.6,0.000033
56.52,0.000033


In [ ]:
# Replace '$-' with NaN
df.replace('$- ', np.nan, inplace=True)

# Display the updated DataFrame
df['Lab Cost'].unique()

array([10, ' $-   ', 92.5, 120, 40, 129.8, 11, 28.04, 74, 25, 88.04, 62,
       111.04, 220, 38, 65, 35.4, 90, 105, 58.5, 6, 60, 50, 28, 92, 38.04,
       113, 27, 80, 45, 125, 36, 59, 35, 75, 8, 115, 30, 30.01, 95, 23,
       215, 49.56, 55, 87, 57, 15, 104, 32, 18, 70, 20, 100, 85, 78.5, 67,
       66.5, 48.8, 96, 160, 34, 166.9, 71.84, 76, 12, 58, 84, 26.05, 29,
       86.25, 78, 97.56, 5, 8.48, 48.32, 26, 36.04, 52, 13, 48, 97, 170,
       123, 7.42, 46, 53, 51.75, 8.8, 47.08, 58.25, 51, 41.73, 102.75,
       128.8, 101, 107, 83, 48.15, 37, 116, 150.04, 183, 47.8, 56.04,
       84.6, 68, 74.9, 135, 81.8, 187, 39.04, 47, 137, 130, 54, 45.5, 210,
       140, 185, 330, 118, 59.5, 153.79, 188, 110, 192, 108, 370, 58.04,
       61, 33, 80.25, 212, 150, 46.03, 43.9, 56, 157, 127, 63, 142, 12.5,
       24, 99, 270, 151, 18.97, 300, 42, 68.04, 109, 22, 88, 180, 26.21,
       30.75, 16, 121, 58.8, 98, 76.09, 17.76, 37.5, 73.04, 240, 182, 21,
       225, 63.04, 168, 82, 35.51, 44.9, 102, 206

In [ ]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.replace('$-', np.nan, inplace=True)


<ipython-input-25-64b2a541cf39>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
<ipython-input-25-64b2a541cf39>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('$-', np.nan, inplace=True)


In [ ]:
df['Lab Cost'].unique()

array([ 10.  ,    nan,  92.5 , 120.  ,  40.  , 129.8 ,  11.  ,  28.04,
        74.  ,  25.  ,  88.04,  62.  , 111.04, 220.  ,  38.  ,  65.  ,
        35.4 ,  90.  , 105.  ,  58.5 ,   6.  ,  60.  ,  50.  ,  28.  ,
        92.  ,  38.04, 113.  ,  27.  ,  80.  ,  45.  , 125.  ,  36.  ,
        59.  ,  35.  ,  75.  ,   8.  , 115.  ,  30.  ,  30.01,  95.  ,
        23.  , 215.  ,  49.56,  55.  ,  87.  ,  57.  ,  15.  , 104.  ,
        32.  ,  18.  ,  70.  ,  20.  , 100.  ,  85.  ,  78.5 ,  67.  ,
        66.5 ,  48.8 ,  96.  , 160.  ,  34.  , 166.9 ,  71.84,  76.  ,
        12.  ,  58.  ,  84.  ,  26.05,  29.  ,  86.25,  78.  ,  97.56,
         5.  ,   8.48,  48.32,  26.  ,  36.04,  52.  ,  13.  ,  48.  ,
        97.  , 170.  , 123.  ,   7.42,  46.  ,  53.  ,  51.75,   8.8 ,
        47.08,  58.25,  51.  ,  41.73, 102.75, 128.8 , 101.  , 107.  ,
        83.  ,  48.15,  37.  , 116.  , 150.04, 183.  ,  47.8 ,  56.04,
        84.6 ,  68.  ,  74.9 , 135.  ,  81.8 , 187.  ,  39.04,  47.  ,
      

In [ ]:
df['Financial Class'].dropna(inplace=True)

In [ ]:
df.isna().sum()

,0
Date,0
Medication Revenue,11936
Lab Cost,28565
Consultation Revenue,5576
Doctor Type,0
Financial Class,0
Patient Type,0
Entry Time,0
Post-Consultation Time,0
Completion Time,0


In [ ]:
dropped_df = df.drop(columns=['Patient ID','Patient Type','Date'])
dropped_df

,Medication Revenue,Lab Cost,Consultation Revenue,Doctor Type,Financial Class,Entry Time,Post-Consultation Time,Completion Time
0,1183.22,10.0,20.17,ANCHOR,HMO,08:35:45,09:17:54,09:29:46
1,738.48,NaN,15.00,ANCHOR,INSURANCE,19:19:16,21:02:36,21:24:07
2,660.00,NaN,21.17,ANCHOR,HMO,10:46:52,11:56:25,12:06:28
3,600.00,NaN,NaN,ANCHOR,MEDICARE,09:38:34,10:55:50,10:58:02
4,591.60,NaN,12.00,ANCHOR,INSURANCE,11:16:21,12:06:49,12:06:54
...,...,...,...,...,...,...,...,...
29993,NaN,NaN,NaN,ANCHOR,PRIVATE,14:01:24,14:08:56,14:15:42
29994,NaN,NaN,NaN,ANCHOR,PRIVATE,14:01:33,14:21:17,14:21:53
29995,NaN,NaN,NaN,ANCHOR,PRIVATE,14:01:41,14:13:57,14:16:29
29996,NaN,NaN,NaN,LOCUM,CORPORATE,14:01:42,14:13:21,14:13:22


In [ ]:
df['Doctor Type'].unique()

array(['ANCHOR', 'LOCUM', 'FLOATING'], dtype=object)

### Doctor Roles Definitions

**1. Doctor ANCHOR**  
<span style="color:blue">This term typically refers to a primary or main doctor in a leading or central role within a medical practice or department.</span>

**2. LOCUM**  
<span style="color:green">A locum doctor is a temporary replacement who fills in for a regular doctor when they are unavailable, working in various settings such as hospitals or clinics.</span>

**3. FLOATING**  
<span style="color:purple">A floating doctor moves between different areas or facilities, providing coverage where there are gaps or higher demands, without a permanent assignment.</span>


In [ ]:
label_encoder = LabelEncoder()
dropped_df['doctor_type_encoded'] = label_encoder.fit_transform(df['Doctor Type'])


In [ ]:
df['Financial Class'].unique()

array(['HMO', 'INSURANCE', 'MEDICARE', 'CORPORATE', 'PRIVATE'],
      dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dropped_df['Financial Class_encoded'] = label_encoder.fit_transform(df['Financial Class'])


In [ ]:
dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29998 entries, 0 to 29997
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Medication Revenue       18062 non-null  float64
 1   Lab Cost                 1433 non-null   float64
 2   Consultation Revenue     24422 non-null  float64
 3   Doctor Type              29998 non-null  object 
 4   Financial Class          29998 non-null  object 
 5   Entry Time               29998 non-null  object 
 6   Post-Consultation Time   29998 non-null  object 
 7   Completion Time          29998 non-null  object 
 8   Financial Class_encoded  29998 non-null  int64  
 9   doctor_type_encoded      29998 non-null  int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 2.3+ MB


In [ ]:
dropped_df.drop(columns=['Doctor Type','Financial Class'],inplace=True)

In [ ]:
dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29998 entries, 0 to 29997
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Medication Revenue       18062 non-null  float64
 1   Lab Cost                 1433 non-null   float64
 2   Consultation Revenue     24422 non-null  float64
 3   Entry Time               29998 non-null  object 
 4   Post-Consultation Time   29998 non-null  object 
 5   Completion Time          29998 non-null  object 
 6   Financial Class_encoded  29998 non-null  int64  
 7   doctor_type_encoded      29998 non-null  int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 1.8+ MB


In [ ]:
dropped_df.head()

,Medication Revenue,Lab Cost,Consultation Revenue,Entry Time,Post-Consultation Time,Completion Time,Financial Class_encoded,doctor_type_encoded
0,1183.22,10.0,20.17,08:35:45,09:17:54,09:29:46,1,0
1,738.48,NaN,15.00,19:19:16,21:02:36,21:24:07,2,0
2,660.00,NaN,21.17,10:46:52,11:56:25,12:06:28,1,0
3,600.00,NaN,NaN,09:38:34,10:55:50,10:58:02,3,0
4,591.60,NaN,12.00,11:16:21,12:06:49,12:06:54,2,0


In [ ]:
dropped_df.drop(columns=['Entry Time','Post-Consultation Time','Completion Time'],inplace=True)

In [ ]:
dropped_df.columns

Index(['Medication Revenue', 'Lab Cost', 'Consultation Revenue',
       'Financial Class_encoded', 'doctor_type_encoded'],
      dtype='object')

In [ ]:
X = dropped_df.drop('Financial Class_encoded', axis=1)
y = dropped_df['Financial Class_encoded']


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer



imputer = IterativeImputer(max_iter=10, random_state=42)

# Apply the imputer to the features (X)
X_imputed = imputer.fit_transform(X)

# Convert the imputed data back to a DataFrame (optional for easier use later)
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

# Check the results
print(X_imputed_df.isnull().sum())  # Ensure there are no more missing values


Medication Revenue      0
Lab Cost                0
Consultation Revenue    0
doctor_type_encoded     0
dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_imputed_df, y, test_size=0.2, random_state=42)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
models = {
    "Logistic Regression": LogisticRegression(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "Gradient Boosting": GradientBoostingClassifier()
}


In [ ]:
for name, model in models.items():
    model.fit(X_train_scaled, y_train)  # Train model
    y_pred = model.predict(X_test_scaled)  # Predict on test set

    # Evaluate model
    print(f"{name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"{name} Classification Report:\n{classification_report(y_test, y_pred)}")
    print(f"{name} Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}\n")


Logistic Regression Accuracy: 0.4652
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.30      0.00      0.00      1397
           1       0.67      0.00      0.01       782
           2       0.44      0.84      0.58      1957
           3       0.00      0.00      0.00        61
           4       0.50      0.63      0.56      1803

    accuracy                           0.47      6000
   macro avg       0.38      0.30      0.23      6000
weighted avg       0.45      0.47      0.36      6000

Logistic Regression Confusion Matrix:
[[   3    0 1145    1  248]
 [   4    2  232    0  544]
 [   2    0 1652    2  301]
 [   0    1   24    0   36]
 [   1    0  668    0 1134]]

K-Nearest Neighbors Accuracy: 0.7480
K-Nearest Neighbors Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.44      0.51      1397
           1       0.64      0.68      0.66       782
        

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Gradient Boosting Accuracy: 0.7835
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.30      0.43      1397
           1       0.74      0.83      0.78       782
           2       0.78      0.97      0.86      1957
           3       0.42      0.08      0.14        61
           4       0.82      0.96      0.89      1803

    accuracy                           0.78      6000
   macro avg       0.70      0.63      0.62      6000
weighted avg       0.77      0.78      0.75      6000

Gradient Boosting Confusion Matrix:
[[ 417  193  513    1  273]
 [  71  649    7    6   49]
 [  36   10 1895    0   16]
 [   5   18    1    5   32]
 [  32    7   29    0 1735]]

